# Trace Estimator  
https://doi.org/10.1016/0377-0427(96)00018-0 
## b) Algorithm 1 with Gauss-Radau quadrature

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
#for part e)
from matrices import heat_flow_function, Pei_function, VFH_function, Poisson_function, wathen_ge, Lehmer_function 
import time 
from scipy.linalg import block_diag
from helpers import *

In [2]:
#the Poisson matrix
k=12
A= Poisson_function(k=k)
n=k**2
print("n=",n)
u = np.zeros(n)
u[4] = 1
u = np.random.randn(n)
def f(x):
    return 1/x
tol = 1e-5
L = algorithm_1(A=A, u=u, function=f, maxit=200, epsilon=tol)

print("bounds [U,L]=", L)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(A).dot(u))
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )
print("L<U =", L[1]<L[0] )

n= 144
a= 0.0001
b= 8.0
bounds [U,L]= [56.58868925 56.58837986]
exact value: 56.58856142906353
L<I_ex: True
U>I_ex: True
L<U = True


In [3]:
#the Pei matrix
n=120
A=Pei_function(alpha=5, n=n)
def f(x):
    return 1/x
u = np.zeros(n)
u[1] = 1
#u = np.random.randn(n)
tol = 1e-3
L = algorithm_1(A=A, u=u, function=f, maxit=200, epsilon=tol)

print("bounds [U,L]=", L)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(A).dot(u))
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )
print("L<U =", L[1]<L[0] )

a= 0.0001
b= 125.0
bounds [U,L]= [0.1984 0.1984]
exact value: 0.19840000000000016
L<I_ex: True
U>I_ex: False
L<U = False


In [4]:
n = 5
def tridiag(a, b, c, k1=-1, k2=0, k3=1):
    return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)

vec1 = -1*np.ones(n-1)
vec2 = 2*np.ones(n)
vec3 = -1*np.ones(n-1)
A = tridiag(vec1, vec2, vec3)

u = np.zeros(n)
u[1] = 1
# u = np.random.randn(n)
tol = 1e-2
L = algorithm_1(A=A, u=u, function=f, maxit=200, epsilon=tol)

print("bounds [L, U]=", L)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(A).dot(u))
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("L<I_ex:", L[1]<I_ex )
print("U>I_ex:", L[0]>I_ex )
print("L<U =", L[1]<L[0] )

a= 0.0001
b= 4.0
bounds [L, U]= [1.33333333 1.33333333]
exact value: 1.333333333333333
L<I_ex: False
U>I_ex: True
L<U = False


### e) Numerical experiments (first draft)

In [6]:
import time 
import numpy as np
from scipy.linalg import block_diag

In [7]:
#cell to be rewritten, temporary assignement to A
A=np.eye(3,3) 

**The different matrices they experimented on in the paper:**

In [8]:
#example 1: the heat flow matrix

In [2]:
n=2
nu=3 #some random values for now, nu>0 implies heat_flow_matrix is positive definite which is something we want
heat_flow_matrix=heat_flow_function(nu,n)
heat_flow_matrix

array([[13., -3., -3.,  0.],
       [-3., 13.,  0., -3.],
       [-3.,  0., 13., -3.],
       [ 0., -3., -3., 13.]])

In [17]:
#the Pei matrix
alpha=2
n=4
Pei_matrix=Pei_function(alpha, n)

u = np.random.rand(n)
U, L = algorithm_1(A=Pei_matrix, u=u, function=f, maxit=200, epsilon=tol)
print("L=", L)
print("U=", U)

# exact value of u^T f(A) u:
I_ex = u.dot(np.linalg.inv(Pei_matrix)@u)
#I_ex = u.dot((A)@u)
print("exact value:", I_ex)

# check 
print("is the exact value between L and U:", I_ex<U and L<I_ex)

a = 0.0001
b = 6.0
Tolerance 1e-13 was reached at iteration 2
L= 0.16153600220772452
U= 0.16153600220772452
exact value: 0.19887453815178047
is the exact value between L and U: False


In [ ]:
#running time of algo 2

start=time.time()
E,L,U=algorithm_2(A,m,p) #for some m,p to tune 
execution_algo_2=time.time()-start 

In [4]:
#running time using built in numpy functions
A=heat_flow_matrix
start=time.time()
Tr_A_inv=np.trace(np.linalg.inv(A))
execution_built_in=time.time()-start 
print('execution time', execution_built_in)

execution time 0.0012359619140625


In [6]:
#solving n linear equations

#(Armelle: in my opinion, when they say compute trace(inv(A)) using n linear equations, they mean computing 
#e_i.T inv(A) e_i for i in {1,...,n} )

start=time.time()
n=A.shape[0]

trace=0
for i in range(n):
    e=np.zeros(n)
    e[i]=1
    trace+=e.T@np.linalg.inv(A)@e
execution_linear_equations=time.time()-start 

print('execution time', execution_linear_equations)

execution time 0.03495073318481445


In [ ]:
#using algorithm 1

n=A.shape[0]
start=time.time()
trace=0
for i in range (n):
    trace+= #output of algo 1 (I guess here also parameter values to tune)
execution_algo_1=time.time()-start 